In [1]:
import pandas as pd
from Bio import Phylo
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
import ete3
from Bio import Phylo

In [2]:
file = 'helix1n2.fasta'

In [3]:
d = {}

with open(file) as f:
    for line in f:
        if len(line) > 1:
            if '>' in line:
                key = line.strip()
                d[key] = ""
            else:
                d[key] += line.strip() + "+"

d = {key: d[key][:-1] for key in d}

In [4]:
d

{'>tr|L8H844|L8H844_ACACF/103-122 Ammonium transporter OS=Acanthamoeba castellanii (strain ATCC 30010 / Neff) OX=1257118 GN=ACA1_230500 PE=3 SV=1 L8H844_Helix_1': 'TTAIDTGWLLMSSYLVFWFA',
 '>tr|L8GGA0|L8GGA0_ACACF/24-45 Ammonium transporter OS=Acanthamoeba castellanii (strain ATCC 30010 / Neff) OX=1257118 GN=ACA1_362520 PE=3 SV=1 L8GGA0_Helix_1': 'AYLLVCTALVWFMTPGLAFFYG',
 '>tr|L8GXL2|L8GXL2_ACACF/190-213 Ammonium transporter L8GXL2_Helix_1': 'ANWYTIWVHVAMLVLAGFSFFYTF',
 '>tr|L8GFX3|L8GFX3_ACACF/25-46 Ammonium transporter subfamily protein OS=Acanthamoeba castellanii (strain ATCC 30010 / Neff) OX=1257118 GN=ACA1_362530 PE=3 SV=1 L8GFX3_Helix_1': 'AYILLCTALVWLMTPGLAFFYG',
 '>tr|L8H5K0|L8H5K0_ACACF/57-76 Ammonium transporter subfamily protein OS=Acanthamoeba castellanii (strain ATCC 30010 / Neff) OX=1257118 GN=ACA1_098420 PE=3 SV=1 L8H5K0_Helix_1': 'FFRDICIMIFFGFGFLMAFL',
 '>tr|B0C271|B0C271_ACAM1/63-87 Ammonium transporter OS=Acaryochloris marina (strain MBIC 11017) OX=329726 GN=amtB PE=

In [5]:
new = pd.DataFrame.from_dict(d, orient ='index')
#get alignment file

In [6]:
new = new.reset_index()

In [7]:
new = new.replace('>','',regex=True).astype(str)
#get rid of the >tr| bit to make processing easier

In [8]:
new

,index,0
0,tr|L8H844|L8H844_ACACF/103-122 Ammonium transp...,TTAIDTGWLLMSSYLVFWFA
1,tr|L8GGA0|L8GGA0_ACACF/24-45 Ammonium transpor...,AYLLVCTALVWFMTPGLAFFYG
2,tr|L8GXL2|L8GXL2_ACACF/190-213 Ammonium transp...,ANWYTIWVHVAMLVLAGFSFFYTF
3,tr|L8GFX3|L8GFX3_ACACF/25-46 Ammonium transpor...,AYILLCTALVWLMTPGLAFFYG
4,tr|L8H5K0|L8H5K0_ACACF/57-76 Ammonium transpor...,FFRDICIMIFFGFGFLMAFL
...,...,...
1800,tr|Q9V3T3|Q9V3T3_DROME/55-75 BcDNA.GH03016 OS=...,YPQFQDIQVMIFIGFGFLMTF
1801,tr|M9PEN8|M9PEN8_DROME/55-75 Rh50 isoform B OS...,YPQFQDIQVMIFIGFGFLMTF
1802,tr|Q5BP96|Q5BP96_AEDAE/58-78 Ammonium transpor...,YPHFQDIHVMIFIGFGFLMTF
1803,tr|Q5BP95|Q5BP95_AEDAE/58-75 Ammonium transpor...,YPHFQDIHVMIFIGFGFL


In [9]:
new[['not_sure','Ids']] = new['index'].str.split('/', 1, expand = True)

/var/folders/zf/rmc497690_7600q6zw2jztgr0000gn/T/ipykernel_73988/2861884807.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  new[['not_sure','Ids']] = new['index'].str.split('/', 1, expand = True)


In [10]:
new[['IDS','Ids+more']] = new['Ids'].str.split(' ', 1, expand=True)
#splitting

/var/folders/zf/rmc497690_7600q6zw2jztgr0000gn/T/ipykernel_73988/907113194.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  new[['IDS','Ids+more']] = new['Ids'].str.split(' ', 1, expand=True)


In [11]:
new[['protein','organism']] = new['Ids+more'].str.split('OS=', 1, expand=True)
#splitting

/var/folders/zf/rmc497690_7600q6zw2jztgr0000gn/T/ipykernel_73988/1984513756.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  new[['protein','organism']] = new['Ids+more'].str.split('OS=', 1, expand=True)


In [12]:
new[['Organism','rest']] = new['organism'].str.split('OX=', 1, expand=True)
#splitting

/var/folders/zf/rmc497690_7600q6zw2jztgr0000gn/T/ipykernel_73988/3400227619.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  new[['Organism','rest']] = new['organism'].str.split('OX=', 1, expand=True)


In [13]:
new[['eyeds','other']] = new['not_sure'].str.split('|', 1, expand=True)
#splitting

/var/folders/zf/rmc497690_7600q6zw2jztgr0000gn/T/ipykernel_73988/2299190608.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  new[['eyeds','other']] = new['not_sure'].str.split('|', 1, expand=True)


In [14]:
 new[['oother','ooother']] = new['other'].str.split('|', 1, expand=True)
#splitting

/var/folders/zf/rmc497690_7600q6zw2jztgr0000gn/T/ipykernel_73988/3552260513.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  new[['oother','ooother']] = new['other'].str.split('|', 1, expand=True)


In [15]:
new

,index,0,not_sure,Ids,IDS,Ids+more,protein,organism,Organism,rest,eyeds,other,oother,ooother
0,tr|L8H844|L8H844_ACACF/103-122 Ammonium transp...,TTAIDTGWLLMSSYLVFWFA,tr|L8H844|L8H844_ACACF,103-122 Ammonium transporter OS=Acanthamoeba c...,103-122,Ammonium transporter OS=Acanthamoeba castellan...,Ammonium transporter,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_230500 PE=3 SV=1 L8H844_Helix_1,tr,L8H844|L8H844_ACACF,L8H844,L8H844_ACACF
1,tr|L8GGA0|L8GGA0_ACACF/24-45 Ammonium transpor...,AYLLVCTALVWFMTPGLAFFYG,tr|L8GGA0|L8GGA0_ACACF,24-45 Ammonium transporter OS=Acanthamoeba cas...,24-45,Ammonium transporter OS=Acanthamoeba castellan...,Ammonium transporter,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_362520 PE=3 SV=1 L8GGA0_Helix_1,tr,L8GGA0|L8GGA0_ACACF,L8GGA0,L8GGA0_ACACF
2,tr|L8GXL2|L8GXL2_ACACF/190-213 Ammonium transp...,ANWYTIWVHVAMLVLAGFSFFYTF,tr|L8GXL2|L8GXL2_ACACF,190-213 Ammonium transporter L8GXL2_Helix_1,190-213,Ammonium transporter L8GXL2_Helix_1,Ammonium transporter L8GXL2_Helix_1,None,None,None,tr,L8GXL2|L8GXL2_ACACF,L8GXL2,L8GXL2_ACACF
3,tr|L8GFX3|L8GFX3_ACACF/25-46 Ammonium transpor...,AYILLCTALVWLMTPGLAFFYG,tr|L8GFX3|L8GFX3_ACACF,25-46 Ammonium transporter subfamily protein O...,25-46,Ammonium transporter subfamily protein OS=Acan...,Ammonium transporter subfamily protein,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_362530 PE=3 SV=1 L8GFX3_Helix_1,tr,L8GFX3|L8GFX3_ACACF,L8GFX3,L8GFX3_ACACF
4,tr|L8H5K0|L8H5K0_ACACF/57-76 Ammonium transpor...,FFRDICIMIFFGFGFLMAFL,tr|L8H5K0|L8H5K0_ACACF,57-76 Ammonium transporter subfamily protein O...,57-76,Ammonium transporter subfamily protein OS=Acan...,Ammonium transporter subfamily protein,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_098420 PE=3 SV=1 L8H5K0_Helix_1,tr,L8H5K0|L8H5K0_ACACF,L8H5K0,L8H5K0_ACACF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,tr|Q9V3T3|Q9V3T3_DROME/55-75 BcDNA.GH03016 OS=...,YPQFQDIQVMIFIGFGFLMTF,tr|Q9V3T3|Q9V3T3_DROME,55-75 BcDNA.GH03016 OS=Drosophila melanogaster...,55-75,BcDNA.GH03016 OS=Drosophila melanogaster OX=72...,BcDNA.GH03016,Drosophila melanogaster OX=7227 GN=Rh50 PE=2 S...,Drosophila melanogaster,7227 GN=Rh50 PE=2 SV=1 Q9V3T3_Helix_1,tr,Q9V3T3|Q9V3T3_DROME,Q9V3T3,Q9V3T3_DROME
1801,tr|M9PEN8|M9PEN8_DROME/55-75 Rh50 isoform B OS...,YPQFQDIQVMIFIGFGFLMTF,tr|M9PEN8|M9PEN8_DROME,55-75 Rh50 isoform B OS=Drosophila melanogaste...,55-75,Rh50 isoform B OS=Drosophila melanogaster OX=7...,Rh50 isoform B,Drosophila melanogaster OX=7227 GN=Rh50 PE=3 S...,Drosophila melanogaster,7227 GN=Rh50 PE=3 SV=1 Helix_1,tr,M9PEN8|M9PEN8_DROME,M9PEN8,M9PEN8_DROME
1802,tr|Q5BP96|Q5BP96_AEDAE/58-78 Ammonium transpor...,YPHFQDIHVMIFIGFGFLMTF,tr|Q5BP96|Q5BP96_AEDAE,58-78 Ammonium transporter 1 OS=Aedes aegypti ...,58-78,Ammonium transporter 1 OS=Aedes aegypti OX=715...,Ammonium transporter 1,Aedes aegypti OX=7159 GN=Rh50-GP1 PE=2 SV=1 Q5...,Aedes aegypti,7159 GN=Rh50-GP1 PE=2 SV=1 Q5BP96_Helix_1,tr,Q5BP96|Q5BP96_AEDAE,Q5BP96,Q5BP96_AEDAE
1803,tr|Q5BP95|Q5BP95_AEDAE/58-75 Ammonium transpor...,YPHFQDIHVMIFIGFGFL,tr|Q5BP95|Q5BP95_AEDAE,58-75 Ammonium transporter 2 OS=Aedes aegypti ...,58-75,Ammonium transporter 2 OS=Aedes aegypti OX=715...,Ammonium transporter 2,Aedes aegypti OX=7159 GN=Rh50-GP2 PE=2 SV=1 Q5...,Aedes aegypti,7159 GN=Rh50-GP2 PE=2 SV=1 Q5BP95_Helix_1,tr,Q5BP95|Q5BP95_AEDAE,Q5BP95,Q5BP95_AEDAE


In [16]:
# new = new.drop_duplicates('IDS')
# #drop duplicates so that we can merge dataframes

In [17]:
new['thisone'] = new['Organism'] + ' ' + new['protein'] + ' ' + new['oother']

In [18]:
thenames = new['oother'].to_list()

In [19]:
with open('thenames.txt', 'w') as fp:
    for ite in thenames:
        # write each item on a new line
        fp.write("%s\n" % ite)
    print('Done')

Done


In [20]:
new

,index,0,not_sure,Ids,IDS,Ids+more,protein,organism,Organism,rest,eyeds,other,oother,ooother,thisone
0,tr|L8H844|L8H844_ACACF/103-122 Ammonium transp...,TTAIDTGWLLMSSYLVFWFA,tr|L8H844|L8H844_ACACF,103-122 Ammonium transporter OS=Acanthamoeba c...,103-122,Ammonium transporter OS=Acanthamoeba castellan...,Ammonium transporter,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_230500 PE=3 SV=1 L8H844_Helix_1,tr,L8H844|L8H844_ACACF,L8H844,L8H844_ACACF,Acanthamoeba castellanii (strain ATCC 30010 / ...
1,tr|L8GGA0|L8GGA0_ACACF/24-45 Ammonium transpor...,AYLLVCTALVWFMTPGLAFFYG,tr|L8GGA0|L8GGA0_ACACF,24-45 Ammonium transporter OS=Acanthamoeba cas...,24-45,Ammonium transporter OS=Acanthamoeba castellan...,Ammonium transporter,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_362520 PE=3 SV=1 L8GGA0_Helix_1,tr,L8GGA0|L8GGA0_ACACF,L8GGA0,L8GGA0_ACACF,Acanthamoeba castellanii (strain ATCC 30010 / ...
2,tr|L8GXL2|L8GXL2_ACACF/190-213 Ammonium transp...,ANWYTIWVHVAMLVLAGFSFFYTF,tr|L8GXL2|L8GXL2_ACACF,190-213 Ammonium transporter L8GXL2_Helix_1,190-213,Ammonium transporter L8GXL2_Helix_1,Ammonium transporter L8GXL2_Helix_1,None,None,None,tr,L8GXL2|L8GXL2_ACACF,L8GXL2,L8GXL2_ACACF,NaN
3,tr|L8GFX3|L8GFX3_ACACF/25-46 Ammonium transpor...,AYILLCTALVWLMTPGLAFFYG,tr|L8GFX3|L8GFX3_ACACF,25-46 Ammonium transporter subfamily protein O...,25-46,Ammonium transporter subfamily protein OS=Acan...,Ammonium transporter subfamily protein,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_362530 PE=3 SV=1 L8GFX3_Helix_1,tr,L8GFX3|L8GFX3_ACACF,L8GFX3,L8GFX3_ACACF,Acanthamoeba castellanii (strain ATCC 30010 / ...
4,tr|L8H5K0|L8H5K0_ACACF/57-76 Ammonium transpor...,FFRDICIMIFFGFGFLMAFL,tr|L8H5K0|L8H5K0_ACACF,57-76 Ammonium transporter subfamily protein O...,57-76,Ammonium transporter subfamily protein OS=Acan...,Ammonium transporter subfamily protein,Acanthamoeba castellanii (strain ATCC 30010 / ...,Acanthamoeba castellanii (strain ATCC 30010 / ...,1257118 GN=ACA1_098420 PE=3 SV=1 L8H5K0_Helix_1,tr,L8H5K0|L8H5K0_ACACF,L8H5K0,L8H5K0_ACACF,Acanthamoeba castellanii (strain ATCC 30010 / ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,tr|Q9V3T3|Q9V3T3_DROME/55-75 BcDNA.GH03016 OS=...,YPQFQDIQVMIFIGFGFLMTF,tr|Q9V3T3|Q9V3T3_DROME,55-75 BcDNA.GH03016 OS=Drosophila melanogaster...,55-75,BcDNA.GH03016 OS=Drosophila melanogaster OX=72...,BcDNA.GH03016,Drosophila melanogaster OX=7227 GN=Rh50 PE=2 S...,Drosophila melanogaster,7227 GN=Rh50 PE=2 SV=1 Q9V3T3_Helix_1,tr,Q9V3T3|Q9V3T3_DROME,Q9V3T3,Q9V3T3_DROME,Drosophila melanogaster BcDNA.GH03016 Q9V3T3
1801,tr|M9PEN8|M9PEN8_DROME/55-75 Rh50 isoform B OS...,YPQFQDIQVMIFIGFGFLMTF,tr|M9PEN8|M9PEN8_DROME,55-75 Rh50 isoform B OS=Drosophila melanogaste...,55-75,Rh50 isoform B OS=Drosophila melanogaster OX=7...,Rh50 isoform B,Drosophila melanogaster OX=7227 GN=Rh50 PE=3 S...,Drosophila melanogaster,7227 GN=Rh50 PE=3 SV=1 Helix_1,tr,M9PEN8|M9PEN8_DROME,M9PEN8,M9PEN8_DROME,Drosophila melanogaster Rh50 isoform B M9PEN8
1802,tr|Q5BP96|Q5BP96_AEDAE/58-78 Ammonium transpor...,YPHFQDIHVMIFIGFGFLMTF,tr|Q5BP96|Q5BP96_AEDAE,58-78 Ammonium transporter 1 OS=Aedes aegypti ...,58-78,Ammonium transporter 1 OS=Aedes aegypti OX=715...,Ammonium transporter 1,Aedes aegypti OX=7159 GN=Rh50-GP1 PE=2 SV=1 Q5...,Aedes aegypti,7159 GN=Rh50-GP1 PE=2 SV=1 Q5BP96_Helix_1,tr,Q5BP96|Q5BP96_AEDAE,Q5BP96,Q5BP96_AEDAE,Aedes aegypti Ammonium transporter 1 Q5BP96
1803,tr|Q5BP95|Q5BP95_AEDAE/58-75 Ammonium transpor...,YPHFQDIHVMIFIGFGFL,tr|Q5BP95|Q5BP95_AEDAE,58-75 Ammonium transporter 2 OS=Aedes aegypti ...,58-75,Ammonium transporter 2 OS=Aedes aegypti OX=715...,Ammonium transporter 2,Aedes aegypti OX=7159 GN=Rh50-GP2 PE=2 SV=1 Q5...,Aedes aegypti,7159 GN=Rh50-GP2 PE=2 SV=1 Q5BP95_Helix_1,tr,Q5BP95|Q5BP95_AEDAE,Q5BP95,Q5BP95_AEDAE,Aedes aegypti Ammon

In [21]:
dic = dict(zip(new['not_sure'],new['thisone']))

In [22]:
len(dic)

1801

In [23]:
dic

{'tr|L8H844|L8H844_ACACF': 'Acanthamoeba castellanii (strain ATCC 30010 / Neff)  Ammonium transporter  L8H844',
 'tr|L8GGA0|L8GGA0_ACACF': 'Acanthamoeba castellanii (strain ATCC 30010 / Neff)  Ammonium transporter  L8GGA0',
 'tr|L8GXL2|L8GXL2_ACACF': nan,
 'tr|L8GFX3|L8GFX3_ACACF': 'Acanthamoeba castellanii (strain ATCC 30010 / Neff)  Ammonium transporter subfamily protein  L8GFX3',
 'tr|L8H5K0|L8H5K0_ACACF': 'Acanthamoeba castellanii (strain ATCC 30010 / Neff)  Ammonium transporter subfamily protein  L8H5K0',
 'tr|B0C271|B0C271_ACAM1': 'Acaryochloris marina (strain MBIC 11017)  Ammonium transporter  B0C271',
 'tr|B0BZN3|B0BZN3_ACAM1': 'Acaryochloris marina (strain MBIC 11017)  Ammonium transporter  B0BZN3',
 'tr|B0C272|B0C272_ACAM1': 'Acaryochloris marina (strain MBIC 11017)  Ammonium transporter  B0C272',
 'tr|A8ZQD5|A8ZQD5_ACAM1': 'Acaryochloris marina (strain MBIC 11017)  Ammonium transporter  A8ZQD5',
 'tr|A0A1A0DHK0|A0A1A0DHK0_ACEPA': 'Acetobacter pasteurianus  Ammonium transport

In [24]:
neu = pd.DataFrame()

In [25]:
neu['Organism'] = new['Organism']

In [26]:
neu = neu.set_index('Organism')

In [27]:
neu

""
Organism
Acanthamoeba castellanii (strain ATCC 30010 / Neff)
Acanthamoeba castellanii (strain ATCC 30010 / Neff)
None
Acanthamoeba castellanii (strain ATCC 30010 / Neff)
Acanthamoeba castellanii (strain ATCC 30010 / Neff)
...
Drosophila melanogaster
Drosophila melanogaster
Aedes aegypti


In [28]:
neu.to_csv('names.csv')

In [29]:
with open('dictionary.txt', 'w') as f:
    print(dic, file=f)

In [30]:
from ete3 import Tree
from Bio import Phylo

In [31]:
tree = Tree('TBE_tree.new')

In [32]:
for leaf in tree:
    for old_name, new_name in dic.items():
        if leaf.name == old_name:
            leaf.name = new_name

In [33]:
print(tree)


   /-Mucilaginibacter paludis DSM 18603  Ammonium transporter  H1YF74
  |
  |--Ktedonobacter racemifer DSM 44963  Ammonium transporter  D6TFE1
  |
  |            /-Desulfuromonas acetoxidans (strain DSM 684 / 11070)  Ammonium transporter  Q1K433
  |           |
  |           |         /-Desulfosporosinus acidiphilus (strain DSM 22704 / JCM 16185 / SJ4)  Ammonium transporter  I4D6Z9
  |         /-|      /-|
  |        |  |   /-|   \-Desulfosporosinus orientis (strain ATCC 19365 / DSM 765 / NCIMB 8382 / VKM B-1628 / Singapore I)  Ammonium transporter  G7WFX2
  |        |  |  |  |
  |        |   \-|   \-Acaryochloris marina (strain MBIC 11017)  Ammonium transporter  A8ZQD5
  |        |     |
  |        |      \-Marichromatium purpuratum 984  Ammonium transporter  W0E077
  |        |
  |        |                                             /-Petrotoga mobilis (strain DSM 10674 / SJ95)  Ammonium transporter  A9BIZ7
  |        |                                          /-|
  |        |     

In [34]:
tree.write(format = 0, outfile = 'TBE_named_tree.new')

In [39]:
free = Tree('RAxML_bipartitions.03_bootstrap.raxml.support')


NewickError: Unexisting tree file or Malformed newick tree structure.
You may want to check other newick loading flags like 'format' or 'quoted_node_names'.

In [ ]:
for leaf in free:
    for old_nom, new_nom in dic.items():
        if leaf.name == old_nom:
            leaf.name = new_nom


In [ ]:
free.write(format = 0, outfile = 'original_bs.new') 

In [35]:
namesed = Tree('TBE_named.new')

In [37]:
import ete3

In [47]:
agree = Tree('TBE_named_tree_ordered_midpoint.tree')

In [44]:
new_size = 2

def make_branches_bigger(node):
    node.img_style["hz_line_width"] = new_size # Change the horizotal lines stroke size
    node.img_style["vt_line_width"] = new_size # Change the vertical lines stroke size
    for c in node.children:
        make_branches_bigger(c)

In [48]:
tf = ete3.TreeStyle()
tf.layout_fn = make_branches_bigger
tf.mode = "c"
tf.show_branch_support = True
tf.show_leaf_name = True


agree.render("TBEbsnamed.pdf", tree_style = tf)

{'nodes': [[18310.470566202264,
   15386.905973348803,
   18314.517584107773,
   15390.952991254311,
   0,
   None],
  [18331.708810620043,
   15357.5302033259,
   18335.71630744781,
   15361.537700153667,
   1,
   None],
  [25799.386827077524,
   15351.943995696185,
   25803.386827077524,
   15355.943995696185,
   2,
   None],
  [18569.30179677004,
   15364.00790909367,
   18573.31677632218,
   15368.022888645808,
   3,
   None],
  [21230.187370989166,
   15479.911186666308,
   21234.273509415936,
   15483.99732509308,
   4,
   None],
  [21632.61566412328,
   15399.059389165499,
   21636.645566708678,
   15403.089291750897,
   5,
   None],
  [22261.366976992376,
   15601.018924874908,
   22265.508519262297,
   15605.16046714483,
   6,
   None],
  [22554.971439084715,
   15473.536456744652,
   22559.03840060655,
   15477.60341826649,
   7,
   None],
  [22713.72081333444,
   15448.596992324254,
   22717.77299403752,
   15452.649173027337,
   8,
   None],
  [23665.756128122495,
   15445.

In [42]:
agree = Tree('03_bootstrap.raxml.bestTree.raxml.support')

In [43]:
agree


Tree node '' (0x118ad74d)

In [37]:
Tree.write(agree, 'retain_bs.new')

'(((((tr|A0A6S5XW90|A0A6S5XW90_ENTCL:0.069738,(((((((((tr|A0A378GLU3|A0A378GLU3_9ENTR:0.017073,tr|A0A3N2SFS2|A0A3N2SFS2_9ENTR:0.00174)0.98:0.049994,tr|A0A2T2Y0D5|A0A2T2Y0D5_9ENTR:0.094347)0.855:0.028738,((tr|A0A085GI50|A0A085GI50_9ENTR:0.009806,tr|A0A381CAY8|A0A381CAY8_9ENTR:0.009353)1:0.061174,tr|A0A4R6E4B1|A0A4R6E4B1_9ENTR:0.042985)1:0.079971)0.554:0.027131,(((tr|A0A4R6ECY1|A0A4R6ECY1_SCAGO:0.08786,tr|A0A3N1I597|A0A3N1I597_9ENTR:0.117906)0.27:0.024891,tr|A0A165BP25|A0A165BP25_9ENTR:0.092478)0.325:0.014252,((tr|A0A318FY98|A0A318FY98_KLEOX:0.052418,(tr|A0A6B8N021|A0A6B8N021_KLEOX:0.064786,tr|A0A0H3H4I7|A0A0H3H4I7_KLEOK:0.039443)0.96:0.032948)0.92:0.024718,tr|A0A4R3ZLI7|A0A4R3ZLI7_9ENTR:0.070971)0.9:0.034218)0.603333:0.026455)0.393333:0.005334,((tr|K8CKZ7|K8CKZ7_CROSK:0.010803,((((tr|A0A423Y4I6|A0A423Y4I6_9ENTR:0.006957,tr|V5U2Z4|V5U2Z4_9ENTR:0.005148)0.82:0.031248,tr|A0A7V7RI44|A0A7V7RI44_CROSK:0.018609)0.625:0.014033,tr|K8C851|K8C851_CROSK:0.003948)0.523333:0.005518,tr|A7MFK3|A7MFK3_C

In [44]:
for leaf in agree:
    for old_nom, new_nom in dic.items():
        if leaf.name == old_nom:
            leaf.name = new_nom

agree.write(format = 0, outfile  = 'retain_bs2.new')

In [41]:
tf = ete3.TreeStyle()
tf.mode = "c"
tf.show_branch_support = True
tf.show_leaf_name = True


agree.render("retain_bs2.pdf", tree_style = tf)

{'nodes': [[16137.016324547028,
   13000.49845289784,
   16141.918367016318,
   13005.40049536713,
   0,
   None],
  [15934.308090096169,
   12432.295052913663,
   15939.732581031585,
   12437.719543849078,
   1,
   None],
  [15210.269459943864,
   11567.401842456613,
   15215.862282852344,
   11572.994665365093,
   2,
   None],
  [13054.596484535941,
   11109.236328360726,
   13059.416787313125,
   11114.05663113791,
   3,
   None],
  [11216.436738675839,
   14681.533098488297,
   11221.664274869852,
   14686.76063468231,
   4,
   None],
  [16379.835369666831,
   13666.038818575125,
   16383.835369666831,
   13670.038818575125,
   5,
   None],
  [15557.983548891745,
   11767.936853224248,
   15563.640399153242,
   11773.593703485743,
   6,
   None],
  [13889.78147067674,
   10972.18469846083,
   13894.099063386388,
   10976.502291170478,
   7,
   None],
  [11036.14291254682,
   12946.46870175009,
   11041.056416439347,
   12951.382205642616,
   8,
   None],
  [15848.436801522046,
   1

In [36]:

print(tree)


               /-Rhodopseudomonas palustris (strain BisA53)  Ammonium transporter  Q07UM1
            /-|
           |   \-Rhodopseudomonas palustris (strain BisB18)  Ammonium transporter  Q21CP1
         /-|
        |  |   /-Rhodopseudomonas palustris (strain ATCC BAA-98 / CGA009)  Ammonium transporter  Q6ND36
      /-|   \-|
     |  |      \-Rhodopseudomonas palustris  Ammonium transporter  A0A323UDG5
   /-|  |
  |  |   \-Rhodopseudomonas palustris  Ammonium transporter  A0A0D7F328
  |  |
  |   \-Enhydrobacter aerosaccus  Ammonium transporter  A0A1T4SFK4
  |
  |               /-Phaeobacter gallaeciensis  Ammonium transporter  A0A366XFS8
  |            /-|
  |         /-|   \-Actibacterium mucosum KCTC 23349  Ammonium transporter  A0A037ZJY5
  |        |  |
  |      /-|   \-Octadecabacter arcticus 238  Ammonium transporter  M9RJS5
  |     |  |
  |     |  |   /-Rhodobacter capsulatus (strain ATCC BAA-309 / NBRC 16581 / SB1003)  Ammonium transporter  D5AU98
  |     |   \-|
  |   /-|   

In [38]:
tree.ladderize()

1364

In [39]:
print(tree)


      /-Enhydrobacter aerosaccus  Ammonium transporter  A0A1T4SFK4
     |
   /-|   /-Rhodopseudomonas palustris  Ammonium transporter  A0A0D7F328
  |  |  |
  |   \-|      /-Rhodopseudomonas palustris (strain BisA53)  Ammonium transporter  Q07UM1
  |     |   /-|
  |     |  |   \-Rhodopseudomonas palustris (strain BisB18)  Ammonium transporter  Q21CP1
  |      \-|
  |        |   /-Rhodopseudomonas palustris (strain ATCC BAA-98 / CGA009)  Ammonium transporter  Q6ND36
  |         \-|
  |            \-Rhodopseudomonas palustris  Ammonium transporter  A0A323UDG5
  |
  |         /-Coraliomargarita akajimensis (strain DSM 45221 / IAM 15411 / JCM 23193 / KCTC 12865 / 04OKA010-24)  Ammonium transporter  D5EP71
  |      /-|
  |     |   \-Maritalea myrionectae  Ammonium transporter  A0A2R4MBA2
  |     |
  |   /-|      /-Octadecabacter arcticus 238  Ammonium transporter  M9RJS5
  |  |  |   /-|
  |  |  |  |  |   /-Phaeobacter gallaeciensis  Ammonium transporter  A0A366XFS8
  |  |  |  |   \-|
  |  |

In [40]:
tree.render("ladderize.pdf")

{'nodes': [[0.5, 729.8092020768717, 4.5, 733.8092020768717, 0, None],
  [4.669285638003274, 18.5, 8.669285638003274, 22.5, 1, None],
  [11.00943235338191, 5.0, 15.00943235338191, 9.0, 2, None],
  [9.726777998819456, 32.0, 13.726777998819456, 36.0, 3, None],
  [14.94951350134154, 17.0, 18.949513501341542, 21.0, 4, None],
  [13.378461847958096, 47.0, 17.378461847958096, 51.0, 5, None],
  [17.264499547568253, 35.0, 21.264499547568253, 39.0, 6, None],
  [21.534156320343218, 29.0, 25.534156320343218, 33.0, 7, None],
  [21.89447909086468, 41.0, 25.89447909086468, 45.0, 8, None],
  [17.01466450095495, 59.0, 21.01466450095495, 63.0, 9, None],
  [20.540462217188747, 53.0, 24.540462217188747, 57.0, 10, None],
  [20.351144573675185, 65.0, 24.351144573675185, 69.0, 11, None],
  [3.999305829787973, 157.3203125, 7.999305829787973, 161.3203125, 12, None],
  [8.653277087746922, 105.5, 12.653277087746922, 109.5, 13, None],
  [12.83491456780557, 83.0, 16.83491456780557, 87.0, 14, None],
  [19.7839772931

In [39]:
dotofrom = pd.DataFrame.from_dict(dic, orient='index', columns=['kingdom'])

In [41]:
dotofrom.to_csv('kingdic.csv')